# Devops News With Ollama

## Preparing the Environment
### Required Libraries
The project requires the installation of the following libraries:

```bash
pip install requests beautifulsoup4 python-dotenv ollama ipykernel ipython
```

In [11]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

### Constants
We define the constants that we will use in the project.

In [12]:
# MODEL = "qwen2.5:3b"
MODEL = "gemma3n:e2b"
WEB_URL = "https://devopscube.com/blog/"

## Clase Website

Define una clase llamada Website que realizará web scraping en un sitio web dado.

Como respuesta, devuelve el título y el contenido del artículo.

In [13]:
class Website:
    """
    Define una clase llamada Website que realizará web scraping en un sitio web dado.
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'No Title Found'
        for irrelevant in soup(['script', 'style', 'img', 'input', 'nav', 'footer', 'header', 'aside']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator='\n', strip=True)
        self.links = [a['href'] for a in soup.find_all('a', href=True)]

In [14]:
# TEST
webscraper = Website(WEB_URL)
print("-" * 40)
print(webscraper.title)
print("-" * 40)
print(webscraper.text)
print("-" * 40)
print(webscraper.links)
print("-" * 40)

----------------------------------------
DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)
----------------------------------------
Blog
COMMON
How to Use Argo CD Image Updater with AWS ECR (Step-by-Step Guide)
In this step by step guide, we will look in to Argo CD Image Updater and how to use it
Aswin Vijayan
Sep 13, 2025
12 min read
AWS
How to Trigger an AWS Lambda Function using an EventBridge Rule
In this guide, we will build an event-driven architecture using AWS EventBridge and AWS Lambda.

You will learn how to
Arun Lal
Sep 12, 2025
7 min read
AWS EKS
How to Use EFS on EKS with the EFS CSI Driver
In this guide, you will learn to provision Amazon EFS as persistent storage for an Amazon EKS cluster. It Includes
Bibin Wilson
,
Arun Lal
Sep 11, 2025
14 min read
Kubernetes
How to Mount OCI Image as Volume in Kubernetes Pods
In this guide, you will learn how to mount OCI container images as volumes in Kubernetes Pods.

This feature is
Aswin Vijayan
Sep 10, 2025
4 min read
AWS
Am

# AI Call Configuration
We are going to use the Ollama library to interact with a local LLM model.

In this section, we define the prompt template and the function to call the model.

### Prompt Template

In [15]:
system_prompt = """
    Eres un asistente que analiza el contenido de un sitio web
    y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación.
    Responder en Markdown.
"""

def user_prompt_for(website):
    user_prompt = f"""
        Estás viendo un sitio web titulado {website.title}
        El contenido de este sitio web es el siguiente;
        proporciona un breve resumen de cada publicación del blog en una tabla en formato Markdown.
        Destaca la noticia y una breve descripción.
        {website.text}
    """
    
    return user_prompt

In [16]:
devops_qube = Website(WEB_URL)
print(user_prompt_for(devops_qube))


        Estás viendo un sitio web titulado DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)
        El contenido de este sitio web es el siguiente;
        proporciona un breve resumen de cada publicación del blog en una tabla en formato Markdown.
        Destaca la noticia y una breve descripción.
        Blog
COMMON
How to Use Argo CD Image Updater with AWS ECR (Step-by-Step Guide)
In this step by step guide, we will look in to Argo CD Image Updater and how to use it
Aswin Vijayan
Sep 13, 2025
12 min read
AWS
How to Trigger an AWS Lambda Function using an EventBridge Rule
In this guide, we will build an event-driven architecture using AWS EventBridge and AWS Lambda.

You will learn how to
Arun Lal
Sep 12, 2025
7 min read
AWS EKS
How to Use EFS on EKS with the EFS CSI Driver
In this guide, you will learn to provision Amazon EFS as persistent storage for an Amazon EKS cluster. It Includes
Bibin Wilson
,
Arun Lal
Sep 11, 2025
14 min read
Kubernetes
How to Mount OCI Image as Volu

### System and User Prompts

In [17]:
prompt_message = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(devops_qube)}
    ]

In [18]:
print(prompt_message)

[{'role': 'system', 'content': '\n    Eres un asistente que analiza el contenido de un sitio web\n    y proporciona un breve resumen, ignorando el texto que podría estar relacionado con la navegación.\n    Responder en Markdown.\n'}, {'role': 'user', 'content': "\n        Estás viendo un sitio web titulado DevOpsCube – Easy DevOps, SRE Guides & Reviews (Page 1)\n        El contenido de este sitio web es el siguiente;\n        proporciona un breve resumen de cada publicación del blog en una tabla en formato Markdown.\n        Destaca la noticia y una breve descripción.\n        Blog\nCOMMON\nHow to Use Argo CD Image Updater with AWS ECR (Step-by-Step Guide)\nIn this step by step guide, we will look in to Argo CD Image Updater and how to use it\nAswin Vijayan\nSep 13, 2025\n12 min read\nAWS\nHow to Trigger an AWS Lambda Function using an EventBridge Rule\nIn this guide, we will build an event-driven architecture using AWS EventBridge and AWS Lambda.\n\nYou will learn how to\nArun Lal\nSe

# Results
Use the Ollama SDK call to get the summary of the website content.

In [20]:
summary = ollama.chat(model=MODEL, messages=prompt_message)

In [21]:
display(Markdown(summary['message']['content']))

| Título de la Publicación | Descripción |
|---|---|
| How to Use Argo CD Image Updater with AWS ECR | Guía paso a paso para utilizar Argo CD Image Updater con AWS ECR. |
| How to Trigger an AWS Lambda Function using an EventBridge Rule | Aprende a construir una arquitectura basada en eventos utilizando AWS EventBridge y AWS Lambda. |
| How to Use EFS on EKS with the EFS CSI Driver | Descubre cómo provisionar Amazon EFS como almacenamiento persistente para un clúster Amazon EKS. |
| How to Mount OCI Image as Volume in Kubernetes Pods | Aprende a montar imágenes OCI como volúmenes en los pods de Kubernetes. |
| Amazon Q CLI: A Practical Guide to Getting Started | Una guía práctica para empezar a usar Amazon Q Developer CLI y evitar escribir comandos largos de AWS CLI. |
| Cloudways Promo Codes & Offers For 2025 | Lista de códigos promocionales y ofertas más recientes de Cloudways, con información detallada sobre las diferentes opciones de gestión de DevOps. |
| Observability as Code with Grafana Git Sync | Explica cómo versionar el código de infraestructura y aplicaciones con Grafana Git Sync para mantener los dashboards de Grafana actualizados. |
| Cloudways Review 2025: Details, Pricing, & Features | Una revisión detallada de Cloudways, cubriendo sus características, precios y ventajas y desventajas. |
| KodeKloud Coupons and Promotions for 2025 | Lista de los últimos cupones y promociones de KodeKloud para obtener descuentos en la suscripción. |
| [55% OFF] Kubernetes Certification Coupon: $1685 Discount On Kubernetes Bundles | Cupón y descuentos para la certificación Kubernetes CKA. |
| Linux Foundation Coupon: 80% Off For DevOps Enthusiasts | Descuentos de hasta el 80% en certificaciones populares como Linux Foundation. |
| [75% Off] Udacity Coupons 2025: Save Big On Nanodegree Programs | Cupones y descuentos de Udacity para ahorrar en los programas NanoDegree. |
| How to Set Up ExternalDNS on EKS? | Aprende a instalar y configurar ExternalDNS en Amazon EKS para automatizar la creación de registros DNS. |
| Become A DevOps Engineer in 2025: [A Practical Roadmap] | Una hoja de ruta práctica para convertirse en ingeniero DevOps en 2025, basada en 10 años de experiencia. |
| Pass the CKA Certification Exam: The Ultimate Study Guide | Guía de estudio completa para prepararse para el examen de certificación Certified Kubernetes Administrator (CKA). |
| Kubernetes Pod Guide: Core Concepts with Practical Examples | Cubre los conceptos básicos de los pods de Kubernetes con ejemplos prácticos y casos de uso. |
| Setup MySQL Operator on Kubernetes (Step by Step Guide) | Guía paso a paso para configurar y operar el operador MySQL en un clúster Kubernetes. |
| Kubernetes DaemonSet: A Comprehensive Guide | Una guía completa para entender los DaemonSets de Kubernetes, ideal para principiantes. |
| KCNA Certification: Detailed Study Guide | Guía de estudio detallada para prepararse para la certificación KCNA (Kubernetes and Cloud Native Associate). |
| KCSA Certification: The Complete Study Guide With Resources | Guía completa con recursos para estudiar para la certificación KCSA (Kubernetes and Cloud Native Security Associate). |
| Automate Adding Clusters to ArgoCD using GitHub Actions | Explica cómo utilizar GitHub Actions para automatizar la adición de clústeres a ArgoCD. |
| How to Dockerize a Golang Application (Step-by-Step Guide) | Tutorial paso a paso para Dockerizar una aplicación Go. |
| Kubernetes Pod Graceful Shutdown with SIGTERM & preStop Hooks | Explica cómo manejar el apagado gracioso de los pods de Kubernetes utilizando SIGTERM y preStop hooks. |
| Kubernetes Architecture: The Definitive Guide (2025) | Una guía completa y detallada de la arquitectura de Kubernetes, desde los componentes del control plane hasta los nodos. |